In [1]:
import glob
from tqdm import tqdm
import pandas as pd

## Postprocess for 3T generated structures

In [2]:
from cif2gro import cif2gro
from match_mol2_coordinates_from_gro import get_mol2_coord

In [3]:
# we first converted all .cif files into .gro files for further analyses
for pro in ['CDK2', 'FXA', 'HSP90']:
    l = sorted([x for x in glob.glob(f'{pro}/3T_processed/*/step*.cif')])
    for i in l:
        ref = '/'.join(i.split('/')[:-1])+'/complex.gro'
        out = i.replace('.cif', '.gro')
        cif2gro(ref, i, out)

In [4]:
# we split the .gro files to get ligand .mol2 files
for pro in ['CDK2', 'FXA', 'HSP90']:
    l = sorted([x for x in glob.glob(f'{pro}/3T_processed/*/*.gro')])
    for i in l:
        mol2_file = f'{pro}/ligand_processed/'+i.split('/')[-2]+'.mol2'
        out = i.replace('.gro', '.mol2')
        get_mol2_coord(mol2_file, i, out)

## Calculate RMSD to initial cocrystal structures

In [5]:
from cal_RMSD import cal_RMSD
from get_energy import get_3T_energy

In [6]:
# pre-defined pockets of the three proteins
pocket_map = {'CDK2': '6-21+29-36+61-68+76-92+125-150',
              'FXA': '41-46+76-90+159-168+171-186+205-216',
              'HSP90': '93-100+101-121+131-144+145-153'}
ref_lig_map = {'CDK2':'1fin', 'FXA': '1ezq', 'HSP90': '1uyg'}

# calculate the RMSD of the #T generated ligand poses to the cocrystal poses
for pro in ['CDK2', 'FXA', 'HSP90']:
    l = sorted([x for x in glob.glob(f'{pro}/3T_processed/*/')])
    for i in l:
        print(f'processing {i}')
        if 'cocry' in i:
            PDB = i.split('/')[-2].split('_')[1]
            cal_RMSD(i,PDB,pro+'/analyses/', pro, pocket_map[pro], cocry=1)
        else:
            PDB = i.split('/')[-2].split('_')[-2]
            cal_RMSD(i,PDB,pro+'/analyses/', pro, pocket_map[pro], cocry=0)

processing CDK2/3T_processed/cocry_1jsv_U55_LIG_dock_1fin_1/
processing CDK2/3T_processed/cocry_2iw8_4SP_LIG_dock_1fin_1/
processing CDK2/3T_processed/dekois2_decoy_1_dock_1fin_1/
processing CDK2/3T_processed/dekois2_ligand_1_dock_1fin_1/
processing CDK2/3T_processed/dude_decoy_1_dock_1fin_1/
processing CDK2/3T_processed/dude_ligand_1_dock_1fin_1/
processing FXA/3T_processed/cocry_2d1j_D01_LIG_dock_1ezq_1/
processing FXA/3T_processed/cocry_2d1j_D01_LIG_dock_1ezq_2/
processing FXA/3T_processed/cocry_2y82_930_LIG_dock_1ezq_1/
processing FXA/3T_processed/dekois2_decoy_1_dock_1ezq_1/
processing FXA/3T_processed/dekois2_ligand_1_dock_1ezq_1/
processing FXA/3T_processed/dude_decoy_1_dock_1ezq_1/
processing FXA/3T_processed/dude_ligand_1_dock_1ezq_1/
processing HSP90/3T_processed/cocry_1byq_ADP_LIG_dock_1uyg_1/
processing HSP90/3T_processed/cocry_2fwy_H64_LIG_dock_1uyg_1/
processing HSP90/3T_processed/cocry_2fwy_H64_LIG_dock_1uyg_2/
processing HSP90/3T_processed/cocry_2fwy_H64_LIG_dock_1uyg_3

In [7]:
# merge RMSD calculation for multiple conformers if available

for pro in ['CDK2', 'FXA', 'HSP90']:
    PATHs = sorted(list(set([x.split(f'dock_{ref_lig_map[pro]}')[0] 
                             for x in glob.glob(f'{pro}/analyses/*_*_RMSD_backbone.csv')])))
    for PATH in PATHs:
        all_files = sorted([x for x in glob.glob(f'{pro}/analyses/*_*_RMSD_backbone.csv')
                          if PATH in x])
        idx = [x.split('/')[-1].split('_')[-3] for x in all_files]
        all_files = [pd.read_csv(x, index_col = 0) for x in all_files]
        all_index = [x.index.tolist() for x in all_files]
        all_index = [[y.replace('.gro', '_'+i) for y in x] 
                     for x,i in zip(all_index, idx)]
        for i,j in zip(all_files, all_index):
            i.index = j
        o = pd.concat(all_files, axis = 0)
        o.to_csv(PATH+f'dock_{ref_lig_map[pro]}_RMSD.csv')

In [41]:
# Here we just show the examples of processing the RMSDs. The raw RMSDs for all 
# ligands mentioned in the paper could be found in the three folder entitled:
# CDK2/CDK2_RMSD_features_raw.tsv
# FXA/FXA_RMSD_features_raw.tsv
# HSP90/HSP90_RMSD_features_raw.tsv

for pro in ['CDK2', 'FXA', 'HSP90']:
    dt = pd.read_csv(f'{pro}/{pro}_RMSD_features_raw.tsv', 
                     index_col = 0, sep = '\t')
    print(f'The shape of {pro} RMSD features is ', dt.shape)

The shape of CDK2 RMSD features is  (36, 308)
The shape of FXA RMSD features is  (36, 106)
The shape of HSP90 RMSD features is  (36, 223)


## Get the minimization energy and vina scores for the 3T minimized structures

In [10]:
# get the 3T minimized energy
for pro in ['CDK2', 'FXA', 'HSP90']:
    path = sorted([x.split(f'dock_{ref_lig_map[pro]}')[0][:-1].split('/')[-1] 
                   for x in glob.glob(f'{pro}/3T_processed/*/')])
    for i in path:
        get_3T_energy(i, pro+'/analyses/', pro+'/3T_processed/', ref_lig_map[pro])

In [11]:
# rescore the 3T minimized structures with smina. We recommended to do this in parallel.

for pro in ['CDK2', 'FXA', 'HSP90']:
    l = sorted([x for x in glob.glob(f'{pro}/3T_processed/*/*.gro')])
    cc = []
    for i in l:
        if 'cocry' in i:
            c = ' '.join(['smina', '-l', i.replace('.gro', '.mol2'), 
                          '-r', i.replace('.gro', '_0.pdb'),
                                '--score_only', '>', i.replace('.gro', '.score'), '\n'])
            cc.append(c)
        else:
            c = ' '.join(['smina', '-l', i.replace('.gro', '.mol2'), 
                          '-r', i.replace('.gro', '.pdb'),
                                '--score_only', '>', i.replace('.gro', '.score'), '\n'])
            cc.append(c)
    f = open(f'{pro}/rescore.sh','w')
    for i in cc:
        f.write(i)
    f.close()

In [13]:
%%bash
cat CDK2/rescore.sh | parallel -j 16 {}
cat FXA/rescore.sh | parallel -j 16 {}
cat HSP90/rescore.sh | parallel -j 16 {}

In [14]:
# merge the rescores into one file in each folder 
for pro in ['CDK2', 'FXA', 'HSP90']:
    PATHs = [x for x in glob.glob(f'{pro}/3T_processed/*/')]   
    for PATH in PATHs:
        files = sorted([x for x in glob.glob(PATH+'*.gro')])
        scores = []
        for i in files:
            try:
                score = [x for x in open(i.replace('.gro', '.score'), 'r') 
                         if x.startswith('Affinity')]
                score = [float(x[:-1].replace('Affinity:','').replace('(kcal/mol)','').strip()) 
                         for x in score][0]
                scores.append(score)
            except:
                files = [x for x in files if x != i]
                print(i, 'failed')
        if scores != []:
            scores = pd.DataFrame(scores)
            idx = [x.split('/')[-1].replace('.gro', '')+'_%d'%(int(PATH.split('/')[-2].split('_')[-1])) 
                   for x in files]
            scores.index = idx
            scores.columns = [PATH.split('/')[-2]]
            scores.to_csv(PATH+'rescore.csv')
        else:
            pass
        
# merge all rescores for a ligand into a file in analyses/
for pro in ['CDK2', 'FXA', 'HSP90']:
    PATHs = sorted(list(set([x.split(f'dock_{ref_lig_map[pro]}')[0][:-1].split('/')[-1]
                    for x in glob.glob(f'{pro}/3T_processed/*/')])))
    for PATH in PATHs:
        p = sorted([x for x in glob.glob(f'{pro}/3T_processed/'+PATH+'_*/rescore.csv')])
        s = [pd.read_csv(x, index_col = 0) for x in p]
        for i in s:
            i.columns = [PATH]
        s = pd.concat(s, axis = 0)
        s.to_csv(f'{pro}/analyses/{PATH}_dock_{ref_lig_map[pro]}_rescore.csv')

## Prepare the features for machine learning

In [15]:
# we first used the 12 vina scores of the first rotamer of a ligand as part of the features
for pro in ['CDK2', 'FXA', 'HSP90']:
    PATHs = sorted([x for x in glob.glob(f'{pro}/analyses/*_rescore.csv')])
    idx = ['complex_1', 'step1_200.xyz_1',
           'step2_200_2000_5.0_16930.xyz_1', 'step2_200_2000_5.0_2177.xyz_1',
           'step2_200_2000_5.0_27766.xyz_1', 'step2_200_2000_5.0_28005.xyz_1',
           'step2_200_2000_5.0_4094.xyz_1', 'step2_200_2000_5.0_47873.xyz_1',
           'step2_200_2000_5.0_77285.xyz_1', 'step2_200_2000_5.0_85412.xyz_1',
           'step2_200_2000_5.0_86398.xyz_1', 'step2_200_2000_5.0_86498.xyz_1']
    files = [pd.read_csv(x, index_col = 0) for x in PATHs]
    files_ = []
    for x in files:
        try:
            files_.append(x.loc[idx])
        except:
            pass
    out = pd.concat(files_, axis = 1)
    out.to_csv(f'{pro}/{pro}_smina_features.tsv', sep = '\t')

In [48]:
# Here we just show the examples of getting smina scores. The raw smina scores for all 
# ligands mentioned in the paper could be found in the three folder entitled:
# CDK2/CDK2_smina_features_raw.tsv
# FXA/FXA_smina_features_raw.tsv
# HSP90/HSP90_smina_features_raw.tsv

for pro in ['CDK2', 'FXA', 'HSP90']:
    dt = pd.read_csv(f'{pro}/{pro}_smina_features_raw.tsv', 
                     index_col = 0, sep = '\t')
    print(f'The shape of {pro} smina features is ', dt.shape)

The shape of CDK2 smina features is  (12, 3765)
The shape of FXA smina features is  (12, 7191)
The shape of HSP90 smina features is  (12, 2452)


In [12]:
import os
import pickle
import pandas as pd

def compile_score(folder,out_tag):
    in_csv = os.path.join(folder,out_tag+'_smina_features_raw.tsv')
    tab = pd.read_csv(in_csv, nrows=1, sep='\t').shape[1]
    comma = pd.read_csv(in_csv, nrows=1, sep=',').shape[1]
    if tab > comma:
        data = pd.read_csv(in_csv,sep='\t',low_memory=False)
    else:
        data = pd.read_csv(in_csv,sep=',',low_memory=False)
    columns = data.columns
    tags = data[ data.columns[0] ]
    tags = [tags.values[i] for i in range(len(tags))]
    complexes = [data.columns[i] for i in range(1, len(data.columns))]
    data = data.values[:,1:]
    out_dict = dict()
    out_dict['complex'] = complexes
    out_dict['dock_score'] = data
    out_dict['tags'] = tags
    pickle.dump(out_dict, open( os.path.join(folder,out_tag+'_3T_score.pkl'), 'wb'))
    return

# Next we compile the smina score values to make them more convenient for subsequent processing
compile_score('CDK2','CDK2')
compile_score('CDK2','CDK2_25A')
compile_score('HSP90','HSP90')
compile_score('HSP90','HSP90_rigid')
compile_score('FXA','FXA')

In [19]:
import numpy as np
import random
import pickle

def merge_features(score_pkl, E_pkl, training_pkl, suffix_tag):
    # score file and energy file has slightly different complex names
    # this difference is suffix_tag, which was previously done manually during raw data generation...
    # we will merge the features while making sure we take care of these different naming conventions
    random_seed = 12345
    data = pickle.load(open(score_pkl,'rb'))
    complexes = data['complex']
    classes = []
    for i in range(len(complexes)):
        words = complexes[i].split('_')
        if words[0] == 'cocry':
            classes.append(1)
        elif words[1] == 'ligand':
            classes.append(1)
        else:
            classes.append(0)
    classes = np.array(classes)
    
    X = data['dock_score'].T
    y = classes
    # Remove nan entries, there are two such columns during the rescoring of CDK2 with 20A pocket
    nan_indices = np.argwhere(X != X)
    excluded_indices = list(set(nan_indices[:,0]))
    included_indices = [i for i in range(X.shape[0])]
    for idx in excluded_indices: 
        included_indices.remove(idx)
    X = X[included_indices,:]
    y = y[included_indices]
    score_complexes = [complexes[i] for i in included_indices]
    
    data = pickle.load(open(E_pkl,'rb'))
    energy_complexes = data['complex']
    energy = data['E_binding']
    seed_list = data['seed']
    indices = []
    for i in range(len(score_complexes)):
        name = score_complexes[i] + suffix_tag
        idx = energy_complexes.index(name)
        indices.append(idx)
    energy_complexes = [energy_complexes[i] for i in indices]
    energy = energy[indices]
    energy = energy.reshape([energy.shape[0],-1])
    
    # Structures with high smina scores or energies occur, so just set these values to 0
    bad_score_idx = np.where(X >= 0)
    bad_score_idx = [(bad_score_idx[0][i],bad_score_idx[1][i]) for i in range(len(bad_score_idx[0]))]
    bad_energy_idx = np.where(energy >= 100)
    bad_energy_idx = [(bad_energy_idx[0][i],bad_energy_idx[1][i]) for i in range(len(bad_energy_idx[0]))]
    for idx in bad_score_idx:
        X[idx[0],idx[1]] = 0
    for idx in bad_energy_idx:
        energy[idx[0],idx[1]] = 0
    X = np.concatenate((X, energy), axis=1)
    
    # Dump merged features to training_pkl after scrambling data indices for classification training
    n = X.shape[0]
    new_idx = [i for i in range(n)]
    random.seed(random_seed)
    random.shuffle(new_idx)
    out = dict()
    out['X'] = X[new_idx,:]
    out['y'] = y[new_idx]
    out['complex_name'] = [score_complexes[i] for i in new_idx]
    pickle.dump(out, open(training_pkl,'wb'))
    
    print(training_pkl,':','(n_sample,n_feature) =',X.shape)
    return

# Create training pickle files from smina score and energy files
E_dir = '../2_structure_generation/Energies/Downsampled_Full_Dataset/'
out_tag = '_scrambled_training_data.pkl'
merge_features('CDK2/CDK2_3T_score.pkl', E_dir+'CDK2_3T_energy.pkl', 'CDK2/CDK2'+out_tag, suffix_tag='_1')
merge_features('CDK2/CDK2_25A_3T_score.pkl', E_dir+'CDK2_25A_3T_energy.pkl', 'CDK2/CDK2_25A'+out_tag, suffix_tag='_dock_1FIN_1')
merge_features('HSP90/HSP90_3T_score.pkl', E_dir+'HSP90_3T_energy.pkl', 'HSP90/HSP90'+out_tag, suffix_tag='_dock_1uyg_1')
merge_features('HSP90/HSP90_rigid_3T_score.pkl', E_dir+'HSP90_rigid_3T_energy.pkl', 'HSP90/HSP90_rigid'+out_tag, suffix_tag='_dock_1uyg_1')
merge_features('FXA/FXA_3T_score.pkl', E_dir+'FXA_3T_energy.pkl', 'FXA/FXA'+out_tag, suffix_tag='_dock_1ezq_1')

print('\nSmina score and energy features successfully merged')

CDK2/CDK2_scrambled_training_data.pkl : (n_sample,n_feature) = (3763, 232)
CDK2/CDK2_25A_scrambled_training_data.pkl : (n_sample,n_feature) = (3764, 232)
HSP90/HSP90_scrambled_training_data.pkl : (n_sample,n_feature) = (2452, 232)
HSP90/HSP90_rigid_scrambled_training_data.pkl : (n_sample,n_feature) = (2452, 232)
FXA/FXA_scrambled_training_data.pkl : (n_sample,n_feature) = (7191, 232)

Smina score and energy features successfully merged
